In [1]:
# read in pkl file
import os
import dill as pickle

# picklefile is the location of the pickled file generated using preader 

# below could be use to import relative to current directory if easier
# cwd = os.getcwd()
# picklefile = cwd + '\\Data\\rdx3.pickle'
picklefile = 'C:\\Users\\eenadbu\\PortableApps\\Git\\PDielec\\Data\\new_TI\\test.pkl'

# This opens the pickled file generated by preader, 
# should read in multiple files if preader prcoessed multiple output files at once 
readers = []   
with open(picklefile,'rb') as f:
     readers = pickle.load(f)

In [2]:
# This reads in information from PDielec and turns it into struct (pymatgen) and atoms (ASE) formats
import PDielec.UnitCell as UnitCell
from pymatgen.core import Lattice, Structure, Molecule
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.io.cif import CifWriter


struct_start = Structure(readers.unit_cells[0].lattice, readers.unit_cells[0].element_names, readers.unit_cells[0].fractional_coordinates)
atoms_start  = AseAtomsAdaptor.get_atoms(struct_start)
struct_end = Structure(readers.unit_cells[-1].lattice, readers.unit_cells[-1].element_names, readers.unit_cells[-1].fractional_coordinates)
atoms_end  = AseAtomsAdaptor.get_atoms(struct_end)

In [3]:
#creates set of cif files of different volumes for QHA calculation

from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import numpy as np


# this creates a list of percentage volume change values you want, in this case 97 to 106 (note not 107) with 1 % step
scale = np.arange(99, 107, 1).tolist()


# sanity check to make sure your going to make atleast 5 structures
if len(scale) < 5:
    Print("you need more than 5 unit cells")
else:
    for i in range(len(scale)):
        # ScaleF is thescaling factor the lattice matric needs to be multiplied by to change the volume by the scale percentage
        ScaleF = (scale[i] / 100) ** (1/3)
        newcell = readers.unit_cells[-1].lattice * ScaleF
        # creates new struct
        struct_new = Structure(newcell, readers.unit_cells[-1].element_names, readers.unit_cells[-1].fractional_coordinates)
        # space group analysis that checks symmetry and makes sure primitive cell is formed, could do some printing/checking to confirm
        syms=SpacegroupAnalyzer(struct_new)
        struct_new = syms.get_primitive_standard_structure()
        #create list of cif filenames
        iter =+ i+1
        name = "Vol-"+ str(iter).zfill(3)
        filename = "%s.cif" % name
        #write symmetrised cif file out into cwd
        writer = CifWriter(struct_new, symprec=0.05)
        writer.write_file(filename)

# just for referance this will print out the space group symbol which maybe useful as a check  
# syms=SpacegroupAnalyzer(struct_end)
# spg_symbol=syms.get_space_group_symbol()
# print(spg_symbol)

In [4]:
#displays starting and final structure to make sure they are different

display(struct_new)
display(struct_end)

Structure Summary
Lattice
    abc : 10.227527969461448 10.227527969461448 10.227527969461448
 angles : 23.70081115788578 23.70081115788578 23.70081115788578
 volume : 151.82929523625668
      A : 10.009549749497944 -2.1002957359481194 0.0
      B : 10.009549749497944 2.1002957359481194 0.0
      C : 9.568846392320154 0.0 3.6110257664959695
PeriodicSite: Bi (11.7565, 0.0000, 1.4348) [0.3973, 0.3973, 0.3973]
PeriodicSite: Bi (17.8315, 0.0000, 2.1762) [0.6027, 0.6027, 0.6027]
PeriodicSite: Se (6.4458, -0.0000, 0.7867) [0.2179, 0.2179, 0.2179]
PeriodicSite: Se (23.1421, -0.0000, 2.8244) [0.7821, 0.7821, 0.7821]
PeriodicSite: Se (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]

Structure Summary
Lattice
    abc : 10.030795753568597 10.030795753573825 10.030795753573825
 angles : 23.700811160569707 23.70081115529865 23.70081115529865
 volume : 143.23518417466875
      A : 2.378562297 -0.0 9.744706504
      B : -1.189281148 2.059895374 9.744706504
      C : -1.189281148 -2.059895374 9.744706504
PeriodicSite: Bi (0.0000, 0.0000, 11.6159) [0.3973, 0.3973, 0.3973]
PeriodicSite: Bi (0.0000, -0.0000, 17.6182) [0.6027, 0.6027, 0.6027]
PeriodicSite: Se (0.0000, 0.0000, 6.3687) [0.2179, 0.2179, 0.2179]
PeriodicSite: Se (0.0000, 0.0000, 22.8654) [0.7821, 0.7821, 0.7821]
PeriodicSite: Se (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]